Fully Homomorphic Encryption
==========================

This is a Fully Homomorphic Encryption system whose security is based on Ring-LWE.
This system is an implementation of the Fan-Vercauteren FHE mechanism using Gentry's bootstrapping

&nbsp;
&nbsp;
&nbsp;

Imports
-----------

In [1]:
import unittest
import random
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler

Helpers
-----------

In [46]:
def param_gen(sec, n):
    '''
    Generates appropriate parameters according to FHE standards
    sec = [128, 192, 256]
    n = [1024, 2048, 4096, 8192, 16384, 32768]
    '''
    table = [[29, 21, 16], [56, 39, 31], [111, 77, 60], [220, 154, 120], [440, 307, 239], [880, 612, 478]]
    k = 0 if sec==128 else 1 if sec==192 else 2
    m = log(n,2)-10
    return table[m][k]

def poly(l):
    '''
    Given a list l composed of elements [l0, l1, l2,...,li], returns the polynomial 'form' of list as:
    l0 * x^i + l1 *x^i-1 + ... + li + x^0
    '''
    poly = 0
    for i in range(0, len(l)):
        poly += l[i]*x^(len(l)-i-1)
    return poly
    

FHE Class
---------

In [47]:
class FHE():
    """
    Constructs a (Leveled) Fully Homomorphic Encryption Enviroment
    
    Constructs a new encryption environment who can be used as
    an LWE_PKE object but also provides addition and multiplication
    on encrypted ciphertexts and has security based on R-LWE
    
    Parameters
    ----------
    sec_lambda : int
        security parameter which defines dimension, defaults to 512
    """
    def __init__(self, sec_lambda=128, n=1024, error_dist=None, t=2):
        k = param_gen(sec_lambda, n)
        self.q, self.t, self.n = randint(2^k, 2^(k+1)), t, n
        self.delta = floor(self.q/self.t)
        
        R.<x> = PolynomialRing(Zmod(self.q))
        self.R = QuotientRing(R, R.ideal(x^n + 1)) #univariate polynomial ring with f(x)=x^n+1 
        
        self.chi = error_dist
        if not error_dist:
            sigma=8/sqrt(2*pi)
            self.chi = DiscreteGaussianDistributionPolynomialSampler(self.R, n, sigma)
        
        self._sk = self._SecretKeyGen()
        self._pk = self._PublicKeyGen(self._sk)
        
    def getPublicKey(self): return self._pk
    
    def getCoeffMod(self): return self.q
    
    def getPlaintextSpace(self): return (self.t, getCoeffMod())
    
    def getCircuitDepth(self): pass #TODO
        
    def _SecretKeyGen(self):
        """
        Generates a (monic polynomial) secret key
        
        Parameters
        ----------
        R : Ring to generate monic key from
        
        Returns
        -------
        secret_key
            a monic polynomial in R
        """
        monic = []
        for i in range(0, self.n):
            monic.append(choice([1, 0, self.q]))
        return self.R(poly(monic))
    
    def _PublicKeyGen(self, sk):
        """
        Generates a public key pair of polynomials
        
        Parameters
        ----------
        sk : secret key to generate public key from
        
        Returns
        -------
        public_key
            a pair of two polynomials in R (a, b) where b is some random polynomial
            and a is: b modified by the secret key and adjusted by some error
        """
        a = self.R.random_element()
        e = self.chi()
        return (self.R(-a*sk+e), a)
    
    def encrypt(self, pk, m):
        """
        Encrypts a plaintext list using public key pk
        
        Parameters
        ----------
        pk : public key to encrypt using
        
        m : plaintext list to encrypt
        
        Returns
        -------
        ciphertext pair
            a pair of two polynomials who are a BFV ciphertext
        """
        m = self.R(poly(m))
        p0, p1 = pk
        
        ZZx = PolynomialRing(ZZ, 'x')
        u = self._SecretKeyGen()
        e1,e2 = self.chi(), self.chi()
        
        a = p0*u + e1 + self.delta*m
        b = p1*u + e2
        return a, b
    
    def decrypt(self, c):
        """
        Decrypt a ciphertext pair using instances secret key
        
        Parameters
        ----------
        c : ciphertext pair to decrypt
        
        Returns
        -------
        list
            plaintext list
        """
        p = c[0]+c[1]*self._sk
        p = p.lift().coefficients(sparse=False)
        nl = []
        for i in p:
            nl.append(round(QQ(self.t)*QQ(i)/QQ(self.q)))
        return list(map(lambda x: x%self.t, nl))[::-1]
    
class FHE_b(FHE):
    '''
    Implements a true fully homomorphic encryption enviroment using bootstrapping
    This class only implements parameter selection and bootstrapping should be done externally.
    '''
    def __init(self):
        log_delta = 1.8/(sec_lambda+110)
        Hf = 1 #for simplicity assume parameterized family x^n+1
        t = 2 #plaintext space
        h = 63 #hamming weight
        alpha, beta = 3.8, 9.2 #with e=2^-64
        d=2^10 #set d=2^k (and q=2^n)
        L_min = ceil(log(t * 2 * (Hf*h + 1) + 0.5, 2))
        

        top = log(4*alpha*beta*t^(L_min-1),2) + (2*L_min+1)*log(d,2)
        bot = 2*sqrt(d*log_delta)
        n=ceil((top/bot)^2)
        self.q = 2^n #coefficient modulus
        
        R.<x> = PolynomialRing(Zmod(self.q))
        self.R = QuotientRing(R, R.ideal(x^d + 1)) #univariate polynomial ring with f(x)=x^n+1 
        
        sigma=ceil((alpha*self.q)/2^(2*sqrt(d*log_delta*n)))      
        self.chi = DiscreteGaussianDistributionPolynomialSampler(R, d, sigma)
        
        self._sk = self.SecretKeyGen()
        self._pk = self.PublicKeyGen(self._sk)
        

In [49]:
fhe = FHE(128)
print(fhe.getCoeffMod())
cipher = fhe.encrypt(fhe.getPublicKey(), [1, 0, 0, 1, 1, 0, 1, 1])
print(cipher[0])
print(cipher[1])
plain = fhe.decrypt(cipher)
print(plain)

589827532
444954763*xbar^1023 + 61631085*xbar^1022 + 146510374*xbar^1021 + 125764314*xbar^1020 + 417342079*xbar^1019 + 373615404*xbar^1018 + 453197421*xbar^1017 + 206487280*xbar^1016 + 138620154*xbar^1015 + 177004395*xbar^1014 + 167165481*xbar^1013 + 365050821*xbar^1012 + 408726008*xbar^1011 + 514287788*xbar^1010 + 547283077*xbar^1009 + 471709992*xbar^1008 + 502010168*xbar^1007 + 342397579*xbar^1006 + 44526680*xbar^1005 + 121770711*xbar^1004 + 509123761*xbar^1003 + 338358666*xbar^1002 + 367132398*xbar^1001 + 374547458*xbar^1000 + 117976673*xbar^999 + 185697063*xbar^998 + 444828551*xbar^997 + 118019820*xbar^996 + 326243346*xbar^995 + 331297185*xbar^994 + 247406263*xbar^993 + 134444185*xbar^992 + 547726158*xbar^991 + 70924803*xbar^990 + 259413061*xbar^989 + 219546436*xbar^988 + 502023413*xbar^987 + 487857070*xbar^986 + 548658932*xbar^985 + 355027567*xbar^984 + 302126958*xbar^983 + 516951279*xbar^982 + 118935831*xbar^981 + 427218160*xbar^980 + 316860322*xbar^979 + 584252978*xbar^978 + 419

Unit Tests
--------------

In [ ]:
class testFHE(unittest.TestCase):
    def test_secretKey(self):
        n = randint(1, 512)
        fhe = FHE(n)
        key = fhe._sk.lift()
        coef = key.coefficients(sparse=False)
        for i in coef:
            if abs(i) not in [1, -1, 0]:
                self.fail("non-monic secret key")
        self.assertLessEqual(key.degree(), n)
        
        

if __name__ == '__main__':
    unittest.main(argv=['-v'], verbosity=2, exit=False)